# 03 Event Study

Compute event windows, lift metrics, and export figures.

In [ ]:
from pathlib import Path
import sys

def find_repo_root():
    cwd = Path().resolve()
    for candidate in (cwd,) + tuple(cwd.parents):
        if (candidate / 'src').is_dir() and (candidate / 'config.yaml').exists():
            return candidate
    return cwd

ROOT = find_repo_root()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import time
import pandas as pd

from src.config import load_config
from src.metrics import build_event_window, add_sale_metrics, add_buckets
from src.sales import add_mechanism_tags
from src.viz import (
    plot_overall_lift_curve,
    plot_lift_curve_by_discount_tier,
    plot_lift_curve_seasonal_vs_nonseasonal,
    plot_decay_by_discount_tier,
    plot_lift_vs_discount_scatter,
)

t0 = time.time()
config = load_config()

panel_path = ROOT / 'data/processed/panel_daily.parquet'
panel = pd.read_parquet(panel_path)
print('loaded panel', len(panel), 'secs', round(time.time() - t0, 1))

sales_path = ROOT / 'data/processed/sales.parquet'
sales = pd.read_parquet(sales_path)
print('loaded sales', len(sales), 'secs', round(time.time() - t0, 1))

sales = add_mechanism_tags(sales, config=config)

event_path = ROOT / 'data/processed/event_window.parquet'
if event_path.exists():
    event_window = pd.read_parquet(event_path)
    print('loaded event_window', len(event_window), 'secs', round(time.time() - t0, 1))
else:
    event_window = build_event_window(panel, sales, config=config)
    event_path.parent.mkdir(parents=True, exist_ok=True)
    event_window.to_parquet(event_path, index=False)
    print('built event_window', len(event_window), 'secs', round(time.time() - t0, 1))

sales = add_sale_metrics(sales, event_window, panel, config=config)
print('added sale metrics', 'secs', round(time.time() - t0, 1))
sales = add_mechanism_tags(sales, config=config)
sales = add_buckets(sales, config=config)
print('added buckets', 'secs', round(time.time() - t0, 1))
sales_path.parent.mkdir(parents=True, exist_ok=True)
sales.to_parquet(sales_path, index=False)

fig_dir = ROOT / 'reports/figures'
fig_dir.mkdir(parents=True, exist_ok=True)
plot_overall_lift_curve(event_window, fig_dir / 'lift_curve_overall.png')
print('wrote', fig_dir / 'lift_curve_overall.png')
plot_lift_curve_by_discount_tier(event_window, sales, fig_dir / 'lift_curve_by_discount_tier.png')
print('wrote', fig_dir / 'lift_curve_by_discount_tier.png')
plot_lift_curve_seasonal_vs_nonseasonal(
    event_window, sales, fig_dir / 'lift_curve_seasonal_vs_nonseasonal.png'
)
print('wrote', fig_dir / 'lift_curve_seasonal_vs_nonseasonal.png')
plot_decay_by_discount_tier(sales, fig_dir / 'decay_by_discount_tier.png')
print('wrote', fig_dir / 'decay_by_discount_tier.png')
plot_lift_vs_discount_scatter(sales, fig_dir / 'lift_vs_discount_scatter.png')
print('wrote', fig_dir / 'lift_vs_discount_scatter.png')

print('Event window rows:', len(event_window))
